In [1]:
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

SEED = 42

np.random.seed (SEED)

In [6]:
from sklearn.model_selection import train_test_split

# Load the dataset
dataset_orig = pd.read_parquet ("./datasets/dataset_clean.parquet")

# Split the dataset
X = dataset_orig.drop(columns=['salary'], axis=1)
y = dataset_orig['salary']
X_train, X_test, y_train, y_test = train_test_split (X,y,test_size=0.2, random_state=SEED)

print ("X train")
print (X_train.info ())
print ()

print ("y train")
print (y_train.info ())
print ()

print ("X test")
print (X_test.info ())
print ()

print ("y test")
print (y_test.info ())
print ()

X train
<class 'pandas.core.frame.DataFrame'>
Index: 5347 entries, 2908 to 860
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   work_location  5347 non-null   object
 1   job_role       5347 non-null   object
 2   name           5347 non-null   object
 3   phone_number   5347 non-null   object
dtypes: object(4)
memory usage: 208.9+ KB
None

y train
<class 'pandas.core.series.Series'>
Index: 5347 entries, 2908 to 860
Series name: salary
Non-Null Count  Dtype  
--------------  -----  
5347 non-null   float64
dtypes: float64(1)
memory usage: 83.5 KB
None

X test
<class 'pandas.core.frame.DataFrame'>
Index: 1337 entries, 3089 to 6652
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   work_location  1337 non-null   object
 1   job_role       1337 non-null   object
 2   name           1337 non-null   object
 3   phone_number   1337 non-null   objec

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Create data preprocessing pipeline
preprocessing = ColumnTransformer([
    ("cat_onehot", OneHotEncoder(handle_unknown="error"), ["work_location", "job_role"]),
], remainder="drop")

dataset_trans = pd.DataFrame(preprocessing.fit_transform (dataset_orig).toarray (), columns=preprocessing.get_feature_names_out())
print (*dataset_trans.columns, sep="\n")

cat_onehot__work_location_Australia
cat_onehot__work_location_Canada
cat_onehot__work_location_China
cat_onehot__work_location_UK
cat_onehot__work_location_USA
cat_onehot__job_role_Account Executive
cat_onehot__job_role_Account Manager
cat_onehot__job_role_Accountant
cat_onehot__job_role_Administrative Assistant
cat_onehot__job_role_Advertising Coordinator
cat_onehot__job_role_Back end Developer
cat_onehot__job_role_Business Analyst
cat_onehot__job_role_Business Development Associate
cat_onehot__job_role_Business Development Manager
cat_onehot__job_role_Business Intelligence Analyst
cat_onehot__job_role_Business Operations Analyst
cat_onehot__job_role_CEO
cat_onehot__job_role_Chief Data Officer
cat_onehot__job_role_Chief Technology Officer
cat_onehot__job_role_Consultant
cat_onehot__job_role_Content Marketing Manager
cat_onehot__job_role_Copywriter
cat_onehot__job_role_Creative Director
cat_onehot__job_role_Customer Service Manager
cat_onehot__job_role_Customer Service Rep
cat_onehot__

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# Create model pipeline
model = pipeline = Pipeline([
    ('data_preprocessing', preprocessing),
    ('random_forest', LinearRegression(random_state=SEED))
])